In [11]:
import datetime
import ifcopenshell
import ifcopenshell.api

In [12]:
def add_task(model, name, predecessor, work_schedule):
    task = ifcopenshell.api.run("sequence.add_task", model, work_schedule=work_schedule)
    task.Name = name
    task.PredefinedType = "CONSTRUCTION"
    task_time = ifcopenshell.api.run("sequence.add_task_time", model, task=task)
    ifcopenshell.api.run(    
        "sequence.edit_task_time", model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2022, 10, 1), "ScheduleDuration": "P1W"}
    )
    if predecessor:
        ifcopenshell.api.run("sequence.assign_sequence", model, relating_process=predecessor, related_process=task) # defaults to an FS relationship
    return task

In [13]:
model = ifcopenshell.open('ifc4_wall_cube.ifc')

In [14]:
workplan = ifcopenshell.api.run("sequence.add_work_plan", model, name="Construction Plan")

In [15]:
schedule = ifcopenshell.api.run("sequence.add_work_schedule", model, name="Construction", work_plan=workplan)

In [16]:
task = add_task(model, "Site est.", None, schedule)
start_task = task # Life is simpler if you know your start task

In [17]:
# Example of building a building wall by wall
for wall in model.by_type("IfcWall"):
    name = f"Construct {wall.Name}"
    task = add_task(model, name, task, schedule)
    ifcopenshell.api.run("sequence.assign_product", model, relating_product=wall, related_object=task)

In [18]:
ifcopenshell.api.run("sequence.cascade_schedule", model, task=start_task) # Notice we didn't care much about explicit dates / etc, you can if you want to, but this'll also autocalculate it all for you.
ifcopenshell.api.run("sequence.recalculate_schedule", model, work_schedule=schedule) # Critical path, float, etc

In [19]:
model.write("ifc4_wall_cube_4d.ifc")